In [3]:
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnableBranch, RunnableSequence
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq


groq_api_key=os.getenv("GROQ_API_KEY")

model=ChatGroq(groq_api_key=groq_api_key,model_name="qwen-2.5-32b")

In [1]:
from pydantic import BaseModel, Field
from typing import Literal
from langchain_core.output_parsers import PydanticOutputParser

In [2]:
class StuOutputCorrectness(BaseModel):
    sentiment: Literal['correct', 'partially_correct', 'incorrect'] = Field(description='Give the sentiment from the student response')
    
parser = PydanticOutputParser(pydantic_object=StuOutputCorrectness)

In [4]:
prompt = PromptTemplate(
    template=(
        "Analyze the following student response and classify its correctness. "
        "Determine whether it is 'correct', 'partially_correct', or 'incorrect'.\n\n"
        "Student Response: {response}\n\n"
        "Provide the classification following this format:\n{format_instruction}"
    ),
    input_variables=['response'],
    partial_variables={
        'format_instruction': parser.get_format_instructions()
    }
)


In [6]:
system_prompt = 'YOu are ai assistant'

In [8]:
from langchain.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", 
         "Analyze the following student response and classify its correctness. "
         "Determine whether it is 'correct', 'partially_correct', or 'incorrect'.\n\n"
         "Student Response: {response}\n\n"
         "Provide the classification following this format:\n{format_instruction}")
    ]
).partial(format_instruction=parser.get_format_instructions())


In [9]:
classifier_chain = prompt | model | parser

In [10]:
classifier_chain.invoke(
    {
        'response': "This sum of 2+2=5"
    }
)

StuOutputCorrectness(sentiment='incorrect')